<a href="https://colab.research.google.com/github/baseerx/Pakistan-Election-Data-Analysis/blob/main/election_analytics_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from google.colab import drive


In [18]:
drive.mount('/content/gdrive')

# Create Spark session
spark = SparkSession.builder \
    .appName("ElectionData") \
    .getOrCreate()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
# Define the file paths
file_path_2002 = '/content/gdrive/MyDrive/Colab Notebooks/Data/National Assembly Result 2002.csv'
file_path_2008 = '/content/gdrive/MyDrive/Colab Notebooks/Data/National Assembly Result 2008.csv'
file_path_2013 = '/content/gdrive/MyDrive/Colab Notebooks/Data/National Assembly Result 2013.csv'
file_path_2018 = '/content/gdrive/MyDrive/Colab Notebooks/Data/National Assembly Results 2018.csv'




In [20]:
# Read the CSV files
df02 = spark.read.csv(file_path_2002, header=True, inferSchema=True)
df08 = spark.read.csv(file_path_2008, header=True, inferSchema=True)
df13 = spark.read.csv(file_path_2013, header=True, inferSchema=True)
df18 = spark.read.csv(file_path_2018, header=True, inferSchema=True)



In [21]:
df02.describe()

DataFrame[summary: string, District: string, Seat: string, Constituency_title: string, Candidate_Name: string, Party: string, Votes: string, Total_Valid_Votes: string, Total_Rejected_Votes: string, Total_Votes: string, Total_Registered_Voters: string, Turnout: string]